In [80]:
! pip install kaggle

In [81]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [82]:
! cp kaggle.json ~/.kaggle/

Устанавливаем права доступа

In [83]:
! chmod 600 ~/.kaggle/kaggle.json

Скачиваем файлы

In [84]:
!kaggle competitions download -c titanic

  0% 0.00/34.1k [00:00<?, ?B/s]
100% 34.1k/34.1k [00:00<00:00, 47.5MB/s]


In [85]:
! unzip "/content/titanic.zip"

Archive:  /content/titanic.zip
replace gender_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: gender_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


Предобработка тренировочных данных

In [86]:
import numpy as np
import pandas as pd

In [87]:
df = pd.read_csv("train.csv")
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


Отделим целевую переменную от датасета

In [88]:
y = df["Survived"]
data = df.drop(columns = ["Survived"])

In [89]:
data.head(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


Оставим только важные фичи

In [90]:
data = data.drop(columns = ["PassengerId","Name","Ticket"])
data.head(10)

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,22.0,1,0,7.2500,NaN,S
1,1,female,38.0,1,0,71.2833,C85,C
2,3,female,26.0,0,0,7.9250,NaN,S
3,1,female,35.0,1,0,53.1000,C123,S
4,3,male,35.0,0,0,8.0500,NaN,S
5,3,male,NaN,0,0,8.4583,NaN,Q
6,1,male,54.0,0,0,51.8625,E46,S
7,3,male,2.0,3,1,21.0750,NaN,S
8,3,female,27.0,0,2,11.1333,NaN,S
9,2,female,14.0,1,0,30.0708,NaN,C


Обработаем пропуски

In [91]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    object 
 2   Age       714 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Cabin     204 non-null    object 
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(3), object(3)
memory usage: 55.8+ KB


In [92]:
fill_value = np.mean(data["Age"])
data["Age"] = data["Age"].fillna(fill_value)

In [93]:
data["Cabin"].value_counts()

B96 B98        4
G6             4
C23 C25 C27    4
C22 C26        3
F33            3
              ..
E34            1
C7             1
C54            1
E36            1
C148           1
Name: Cabin, Length: 147, dtype: int64

In [94]:
data["Cabin"] = data["Cabin"].fillna("B96")

In [95]:
data["Embarked"].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [96]:
data["Embarked"] = data["Embarked"].fillna("S")

In [97]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    object 
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Cabin     891 non-null    object 
 7   Embarked  891 non-null    object 
dtypes: float64(2), int64(3), object(3)
memory usage: 55.8+ KB


Обработаем текстовые данные

In [98]:
data.head(10)

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,22.000000,1,0,7.2500,B96,S
1,1,female,38.000000,1,0,71.2833,C85,C
2,3,female,26.000000,0,0,7.9250,B96,S
3,1,female,35.000000,1,0,53.1000,C123,S
4,3,male,35.000000,0,0,8.0500,B96,S
5,3,male,29.699118,0,0,8.4583,B96,Q
6,1,male,54.000000,0,0,51.8625,E46,S
7,3,male,2.000000,3,1,21.0750,B96,S
8,3,female,27.000000,0,2,11.1333,B96,S
9,2,female,14.000000,1,0,30.0708,B96,C


In [99]:
from sklearn.preprocessing import LabelEncoder
data[["Sex","Cabin","Embarked"]] = data[["Sex","Cabin","Embarked"]].apply(LabelEncoder().fit_transform)


In [100]:
data.head(10)

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,1,22.000000,1,0,7.2500,47,2
1,1,0,38.000000,1,0,71.2833,82,0
2,3,0,26.000000,0,0,7.9250,47,2
3,1,0,35.000000,1,0,53.1000,56,2
4,3,1,35.000000,0,0,8.0500,47,2
5,3,1,29.699118,0,0,8.4583,47,1
6,1,1,54.000000,0,0,51.8625,130,2
7,3,1,2.000000,3,1,21.0750,47,2
8,3,0,27.000000,0,2,11.1333,47,2
9,2,0,14.000000,1,0,30.0708,47,0


Обучаем модель

In [101]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [102]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

KNeighbors

In [103]:
from sklearn.neighbors import KNeighborsClassifier

In [104]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(data,y)
knn_preds = knn.predict(data)
knn_preds_proba = knn.predict_proba(data)[:,1]
print("accuracy",accuracy_score(y,knn_preds))
print("roc_auc",roc_auc_score(y,knn_preds_proba))

accuracy 0.8125701459034792
roc_auc 0.8942228826468113


In [105]:
cross_val_scores = cross_val_score(knn,data,y,cv=5,scoring="roc_auc")
print(cross_val_scores)
print(np.mean(cross_val_scores))

[0.66732543 0.70822193 0.74197861 0.77967914 0.80162212]
0.7397654464113672


Logistic Regression

In [106]:
from sklearn.linear_model import LogisticRegression

In [107]:
lr = LogisticRegression()
lr.fit(data,y)
lr_preds = lr.predict(data)
lr_preds_proba = lr.predict_proba(data)[:,1]
print("accuracy",accuracy_score(y,lr_preds))
print("roc_auc",roc_auc_score(y,lr_preds_proba))

accuracy 0.8103254769921436
roc_auc 0.8591138593295625


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [108]:
cross_val_scores = cross_val_score(lr,data,y,cv=5,scoring="roc_auc")
print(cross_val_scores)
print(np.mean(cross_val_scores))

[0.84940711 0.83161765 0.85755348 0.82600267 0.87721048]
0.8483582777492185


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [109]:
lr.coef_

array([[-9.64183988e-01, -2.62335591e+00, -3.80060009e-02,
        -3.20546102e-01, -9.39610000e-02,  2.33258955e-03,
         8.42846899e-03, -2.80425478e-01]])

Gradient boosting

In [110]:
from sklearn.ensemble import GradientBoostingClassifier

In [111]:
gbdt = GradientBoostingClassifier()
gbdt.fit(data,y)
gbdt_preds = gbdt.predict(data)
gbdt_preds_proba = gbdt.predict_proba(data)[:,1]
print('accuracy', accuracy_score(y, gbdt_preds))
print('roc_auc', roc_auc_score(y, gbdt_preds_proba))

accuracy 0.9023569023569024
roc_auc 0.9445775945632144


In [112]:
cross_val_scores = cross_val_score(gbdt, data, y, cv=5, scoring='roc_auc')
print(cross_val_scores)
print(np.mean(cross_val_scores))

[0.86040843 0.81196524 0.91163102 0.85367647 0.91051722]
0.8696396755750564


In [113]:
gbdt = GradientBoostingClassifier()
kfold = KFold(n_splits=5, shuffle=True, random_state=123)

In [114]:
param_grid = {
    "max_depth": [2, 3],
    "n_estimators": [50, 100, 150]
}

In [115]:
CV_gbdt = GridSearchCV(estimator=gbdt, param_grid=param_grid,scoring="roc_auc", cv = kfold, verbose= 1000)

In [116]:
CV_gbdt.fit(data,y)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5; 1/6] START max_depth=2, n_estimators=50................................
[CV 1/5; 1/6] END .max_depth=2, n_estimators=50;, score=0.904 total time=   0.1s
[CV 2/5; 1/6] START max_depth=2, n_estimators=50................................
[CV 2/5; 1/6] END .max_depth=2, n_estimators=50;, score=0.856 total time=   0.1s
[CV 3/5; 1/6] START max_depth=2, n_estimators=50................................
[CV 3/5; 1/6] END .max_depth=2, n_estimators=50;, score=0.848 total time=   0.1s
[CV 4/5; 1/6] START max_depth=2, n_estimators=50................................
[CV 4/5; 1/6] END .max_depth=2, n_estimators=50;, score=0.850 total time=   0.1s
[CV 5/5; 1/6] START max_depth=2, n_estimators=50................................
[CV 5/5; 1/6] END .max_depth=2, n_estimators=50;, score=0.885 total time=   0.1s
[CV 1/5; 2/6] START max_depth=2, n_estimators=100...............................
[CV 1/5; 2/6] END max_depth=2, n_estimators=100;,

GridSearchCV(cv=KFold(n_splits=5, random_state=123, shuffle=True),
             estimator=GradientBoostingClassifier(),
             param_grid={'max_depth': [2, 3], 'n_estimators': [50, 100, 150]},
             scoring='roc_auc', verbose=1000)

Лучшие параметры

In [117]:
CV_gbdt.best_params_

{'max_depth': 3, 'n_estimators': 150}

In [118]:
CV_gbdt.best_score_

0.877370488105862

Предобработка тестовых данных

In [119]:
data_test = pd.read_csv("test.csv")
data_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [120]:
passenger_id = data_test["PassengerId"]

In [121]:
data_test = data_test.drop(columns = ["PassengerId", "Name", "Ticket"])
data_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,34.5,0,0,7.8292,NaN,Q
1,3,female,47.0,1,0,7.0000,NaN,S
2,2,male,62.0,0,0,9.6875,NaN,Q
3,3,male,27.0,0,0,8.6625,NaN,S
4,3,female,22.0,1,1,12.2875,NaN,S


In [132]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    int64  
 2   Age       332 non-null    float64
 3   SibSp     418 non-null    int64  
 4   Parch     418 non-null    int64  
 5   Fare      417 non-null    float64
 6   Cabin     418 non-null    int64  
 7   Embarked  418 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 26.2 KB


In [133]:
data_test['Age'] = data_test['Age'].fillna(np.mean(data_test["Age"]))

In [134]:
data_test['Fare'] = data_test['Fare'].fillna(np.mean(data['Fare']))

In [135]:
data_test['Embarked'] = data_test['Embarked'].fillna('S')

In [136]:
data_test["Cabin"] = data_test["Cabin"].fillna("B57")

In [137]:
data_test[["Sex","Cabin","Embarked"]] = data_test[["Sex","Cabin","Embarked"]].apply(LabelEncoder().fit_transform)

In [139]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    int64  
 2   Age       418 non-null    float64
 3   SibSp     418 non-null    int64  
 4   Parch     418 non-null    int64  
 5   Fare      418 non-null    float64
 6   Cabin     418 non-null    int64  
 7   Embarked  418 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 26.2 KB


Предсказываем 3  методами

In [140]:
y_pred_lr = lr.predict(data_test)
y_pred_knn = knn.predict(data_test)
y_pred_gbdt = CV_gbdt.predict(data_test)

Итоговый результат

In [141]:
y_pred_gbdt = pd.DataFrame(y_pred_gbdt, columns=['Survived'])
y_pred_gbdt['PassengerId'] = passenger_id
y_pred_gbdt = y_pred_gbdt[['PassengerId', 'Survived']]
y_pred_gbdt.to_csv('submission_gbdt.csv', index=None)
y_pred_gbdt.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
